<a href="https://colab.research.google.com/github/rohithub55/airbnb-analysis/blob/main/airbnb_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIRBNB ANALYSIS

#  Since 2008, guests and hosts have used Airbnb to expand on traveling possibilities and present a more unique, personalized way of experiencing the world. Today, Airbnb became one of a kind service that is used and recognized by the whole world. Data analysis on millions of listings provided through Airbnb is a crucial factor for the company. These millions of listings generate a lot of data - data that can be analyzed and used for security, business decisions, understanding of customers' and providers' (hosts) behavior and performance on the platform, guiding marketing initiatives, implementation of innovative additional services and much more.


# This dataset has around 49,000 observations in it with 16 columns and it is a mix between categorical and numeric values.

In [1]:
#Let's fuel up the rocket to launch...
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.express as px

# Mount the google drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Let's load the fuel (Dataset)

In [3]:
#creating directory path
dir_path='/content/drive/My Drive/Dataset'

In [4]:
#read dataset and naming it as airbnb data frame
airbnb_df=pd.read_csv("/content/drive/MyDrive/Airbnb/Copy of Airbnb NYC 2019.csv")